## <span style="color:red">Classi astratte e loro uso</span>

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class classe_astratta(ABC):
    '''classe_astratta è astratta perché definisce (almeno) un metodo
        astratto
    '''
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        '''Un metodo astratto si 'riconosce' dal decoratore @abstractmethod'''
        pass

#### Non possono esistere oggetti della classe astratta

In [ ]:
A = classe_astratta(10) #In altri termini, una classe astratta non si può istanziare

#### Sono le eventuali sottoclassi che devono implementare i metodi astratti

In [ ]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        pass

In [ ]:
S = sottoclasse(10)

In [ ]:
S.un_metodo_astratto()

In [ ]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [ ]:
S = sottoclasse(10)

In [ ]:
S.un_metodo_astratto()

#### Una sottoclasse di una classe astratta deve implementare ***tutti*** i metodi astratti

In [ ]:
class classe_astratta(ABC):
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        pass
    
    @abstractmethod
    def un_secondo_metodo_astratto(self):
        pass

In [ ]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [ ]:
S = sottoclasse(10)

#### I metodi di una classe astratta devono quindi essere implementati tutti da "qualche" sottoclasse, non necessariamente dalla stessa

In [ ]:
class classe_astratta(ABC):
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        pass
    
    @abstractmethod
    def un_secondo_metodo_astratto(self):
        pass

In [ ]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [ ]:
class sotto_sottoclasse(sottoclasse):
    def un_secondo_metodo_astratto(self):
        pass

In [ ]:
SS = sotto_sottoclasse(20)

In [ ]:
SS.un_metodo_astratto()

In [ ]:
SS.un_secondo_metodo_astratto()

## A che cosa serve tutto ciò?

Una classe astratta viene tipicamente utilizzata per definire funzionalità default per le sue sottoclassi. Il concetto è simile ma non coincide con quello noto di __interfaccia__, in quanto (a differenza delle interfacce) le classi astratte possono essere parzialmente implementate.

Un caso tipico di utilizzo si ha quando le sottoclassi (che estendono la classe astratta) hanno più attributi e metodi comuni.

In [ ]:
from abc import ABC, abstractmethod
import math
import pygame

# Some colors
black = (0,0,0)
white = (255,255,255)
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)

class mydisplay:
    '''Classe che definisce un display (una finestra) dove potranno essere
       disegnati  "singoli" oggetti geometrici. La classe utilizza il package
       pygame'''
    
    def __init__(self, width=800, height=600):
        pygame.display.init()
        self.width = width      # Size of
        self.height = height    # display
        self.display = pygame.display.set_mode((width,height))
    
    @staticmethod
    def close():
        pygame.quit()
        
    def clean(self):
        '''Pulisce il display'''
        self.display.fill(white)
    
    def drawline(self, p, q, color=black):
        '''Disegna un segmento'''
        self.clean()
        pygame.draw.line(self.display, color, (p[0], self.height-p[1]), \
                         (q[0],self.height-q[1]), 3)
        pygame.display.update()
        
    def drawcircle(self, c, r, color=black):
        '''Disegna un cerchio (pieno)'''
        self.clean()
        pygame.draw.circle(self.display, color, (c[0],self.height-c[1]), r)
        pygame.display.update()
        
    def drawrect(self, bl, w, h, color=black):
        '''Disegna un rettangolo'''
        self.clean()
        pygame.draw.rect(self.display, color, \
                         pygame.Rect(bl[0],self.height-bl[1]-h, w, h))
        pygame.display.update()    
    
class oggetto_geometrico(ABC):
    '''Classe che definisce un oggetto astratto generico.
       Opportune sottoclassi dovranno fornire implementazioni
       per le funzioni di spostamento e rotazione dell'oggetto.
       La classe definisce e implementa un metodo per la rotazione
       di un vettore con applicazione nell'origine (quindi definito
       solo dalle coordinate del punto terminale)
    '''
    def __init__(self, display, color=black):
        '''Inizializza un display pygame di dimensioni (width)x(height).
        Il parametro colore è il colore con cui verrà disegnato l'oggetto  '''
        self.display = display
        self.color = color
    
    @staticmethod
    def ruotaV(x, y, theta):
        '''Rotazione di un vettore con punto di applicazione l'origine'''
        xp = x*math.cos(theta)-y*math.sin(theta)
        yp = x*math.sin(theta)+y*math.cos(theta)
        return xp,yp
        
    @abstractmethod
    def sposta(self, dx, dy):
        '''Definisce uno spostamento dell'oggetto di dx punti lungo
           l'asse x e di dy punti lungo l'asse y'''
        pass
    
    @abstractmethod
    def ruota(self, theta):
        '''Definisce la rotazione dell'oggetto di un angolo theta radianti'''
        pass

In [ ]:
class line(oggetto_geometrico):
    
    def __init__(self, p, q, display, color=black):
        '''Initialize a pygame display '''
        super().__init__(display, color)
        self.p = p
        self.q = q
        self.display.drawline(p,q,color)
    
    def sposta(self, dx, dy, disegna=True):
        '''Implementa lo spostamento rigido di un segmento'''
        from operator import add
        self.p = tuple(map(add,self.p,(dx,dy)))
        self.q = tuple(map(add,self.q,(dx,dy)))
        disegna and self.display.drawline(self.p,self.q,self.color)
        
    def ruota(self, theta):
        '''Implementa la rotazione di un segmento'''
        cx = (self.p[0]+self.q[0])/2
        cy = (self.p[1]+self.q[1])/2
        self.sposta(-cx,-cy, disegna=False)
        self.p = self.ruotaV(*self.p, theta)
        self.q = self.ruotaV(*self.q, theta)
        self.sposta(cx,cy)

In [ ]:
D = mydisplay()

In [ ]:
l = line((0,0),(100,200),D)

In [ ]:
l.sposta(200,150)

In [ ]:
l.ruota(-math.pi/2)

In [ ]:
class cerchio(oggetto_geometrico):
    
    def __init__(self, c, r, display, color=black):
        super().__init__(display, color)
        self.c = c
        self.r = r
        self.display.drawcircle(c, r, color)
    
    def sposta(self, dx, dy):
        '''Implementa lo spostamento rigido di un cerchio'''
        from operator import add
        self.c = tuple(map(add,self.c,(dx,dy)))
        self.display.drawcircle(self.c, self.r, self.color)
        
    def ruota(self, theta):
        '''La rotazione non ha effetto (perché non identifichiamo
           punti particolari sul cerchio)'''
        pass

In [ ]:
c = cerchio((200,200), 50, D)

In [ ]:
c.sposta(100,300)

In [ ]:
c.ruota(1)

In [ ]:
class rettangolo(oggetto_geometrico):
    
    def __init__(self, p, w, h, display, color=black):
        super().__init__(display, color)
        self.bottomleft = p
        self.w = w
        self.h = h
        self.display.drawrect(p, w, h, color)
    
    def sposta(self, dx, dy, disegna=True):
        '''Implementa lo spostamento rigido di un rettangolo'''
        from operator import add
        self.bottomleft = tuple(map(add,self.bottomleft,(dx,dy)))
        disegna and self.display.drawrect(self.bottomleft, self.w, self.h, self.color)
        
    def ruota(self, theta):
        from operator import add
        assert theta in {math.pi/2, -math.pi/2}
        cx = self.bottomleft[0]+self.w/2
        cy = self.bottomleft[1]+self.h/2
        self.sposta(-cx,-cy, disegna=False)
        self.bottomleft = self.ruotaV(*self.bottomleft, theta)
        if theta > 0:
            shift = (-self.h,0)
        else:
            shift = (0, -self.w)
        self.bottomleft = tuple(map(add,self.bottomleft,shift))
        self.w, self.h = self.h, self.w
        self.sposta(cx,cy)

In [ ]:
r = rettangolo((1,1), 150, 70, D, color=(124,110,87))

In [ ]:
r.sposta(100,100)

In [ ]:
r.ruota(math.pi/2)

In [ ]:
r.ruota(-math.pi/2)

In [ ]:
D.close()

#### Un (relativamente) semplice progetto

Modificare la classe mydisplay in modo che supporti contemporaneamente le operazioni su più oggetti geometrici